# models

> Model and architecture tooling

In [ ]:
#|default_exp models

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from isaacai.utils import *
from isaacai.dataloaders import *
from matplotlib import pyplot as plt

from fastcore.all import *
import torch
from torch import nn
from torch import Tensor
from datasets import load_dataset
from torch.utils.data import DataLoader
import pandas as pd 
import numpy as np
from datasets import Dataset

In [ ]:
dls = DataLoaders(*load_fashion_mnist(sample=200))
dls.x_name,dls.y_name = 'image','label'

Found cached dataset fashion_mnist (/home/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/8d6c32399aa01613d96e2cbc9b13638f359ef62bb33612b077b4c247f6ef99c1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#| export models
class SimpleNet(nn.Module):
    '''simple net'''
    def __init__(self,in_features,hidden_features, out_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features,hidden_features)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_features,out_features)

    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
model = SimpleNet(784,64,10)
test_eq(model(first(dls.train)['image']).shape,(32,10))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()